In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
# Configure Matplotlib to use LaTeX for text rendering
plt.rcParams['text.usetex'] = True

# Set fonts for Matplotlib
plt.rcParams['font.serif'] = 'Palatino'
plt.rcParams['font.sans-serif'] = 'Helvetica'
plt.rcParams['font.monospace'] = 'Courier'
plt.rcParams['font.size'] = 11
width_cm = 15
height_cm = 8
# Convert centimeters to inches
width_in = width_cm / 2.54
height_in = height_cm / 2.54
import seaborn as sns
sns.set_theme()
figsize= (width_in, height_in)



import os
import numpy as np
import pandas as pd 
import wandb
from metrics.plotting_utils import save_plot, get_history_from_project, save_csv, get_dataset_name_mapping
from ood_detection.config import Config
from datasets.config import DATASETS_DICT
api = wandb.Api()
frame_template = "thesis-far-ood-{}"

# Create Plots for Far OOD
1. 
2. 
3. Per Temp: Mean per Dataset + std, Min per Dataset (+ distance), Max per Dataset (+ distance)
4. Top X + Distances, Flop X + Distances
5. Full plot of all for appendix

In [ ]:
# raise ValueError # not execute, needs time
def prep_far_ood(run):
    download_template = 'thesis-far-ood-{}'
    download_name = download_template.format(run)
    working_df = get_history_from_project(download_name, set_name_index=False)
    working_df['ID'] = working_df.name.apply(lambda x: x.split("-")[0])
    working_df['OOD'] = working_df.name.apply(lambda x: x.split("-")[1])
    working_df = working_df.rename(columns={'AUROC':run.upper()}).set_index('name', drop=True)
    return working_df

def get_grouped_auroc_df(df, grouper, metric):
    if metric == 'mean':
        return df.groupby(grouper).mean(numeric_only = True)
    if metric == 'max':
        return df.groupby(grouper).max(numeric_only = True)
    if metric == 'min':
        return df.groupby(grouper).min(numeric_only = True)

def load_and_group(run, grouper, metric):
    df = prep_far_ood(run)
    return get_grouped_auroc_df(df, grouper, metric)

            
def get_all_finished_runs(methods, runs):
    dfs = {}
    for method in methods:
        for run in runs:
            if method == 'mls' and run != '1.0':
                continue
            name = method +'-'+ run
            print(f"Loading {name}")
            try:
                df = prep_far_ood(name)
                assert len(df) == 132
                dfs[name] = df
            except:
                print(f"{name} not finished yet")
                continue
    return dfs

def get_merged_df(df_dict, grouper, metric):
    dfs = []
    for df in df_dict.values():
        dfs.append(get_grouped_auroc_df(df, grouper, metric))
    return pd.concat(dfs, axis=1)

In [ ]:
# collect ...
methods = ['msp', 'mls']
runs = ['0.01', '1.0', '100.0']
dfs = get_all_finished_runs(methods, runs)

In [ ]:
# save grouped plots
groups = ['ID', 'OOD']
strats = ['mean','max','min']
mappings = get_dataset_name_mapping()

for group in groups:
    for strat in strats:
        df = get_merged_df(dfs, group, strat).rename(index= mappings)
        save_csv(df.round(3), '5_1', f'{group}_{strat}')
        save_csv(df.round(3).T, '5_1_T', f'{group}_{strat}')

In [ ]:
mappings = get_dataset_name_mapping()

df = get_merged_df(dfs, 'ID', 'mean').rename(index=mappings)

In [ ]:
mcm_df = df.copy()

In [ ]:
mcm_df.mean(axis=1)

In [ ]:
mcm_df.max(axis=1)

In [ ]:
mcm_df

In [ ]:
fig, ax = plt.subplots(figsize=figsize)

# mean = df[value_column].mean().round(3)
# std = u"\u00B1{})".format(df[value_column+'-std'].mean().round(2))
# label = name + f' (\u03bc= {mean}' + std
value_column='MLS-1.0'
ax.plot(mcm_df[value_column], label=value_column, color='steelblue')
value_column ='MSP-1.0'
ax.plot(mcm_df[value_column], label=value_column,color='mediumseagreen')
value_column = 'MSP-0.01'
ax.plot(mcm_df[value_column], label=value_column, color='mediumvioletred')



ax.hlines(0.5, xmin=-.5, xmax=[len(mcm_df)], colors='darkgrey', alpha=0.9, linestyles='--', lw=2, label='Uninformed Guesser')
ax.vlines([range(len(mcm_df))], ymin=0, ymax=mcm_df.max(axis=1),color='darkgrey', alpha=0.6)

ax.set_ylim(0.2,1.1)
ax.set_xticklabels(mcm_df.index, rotation=45, ha='right')
ax.legend(loc='lower left', fontsize=6)
ax.set_xlabel('ID Dataset')
ax.set_ylabel('mean AUROC')
plt.tight_layout()
plt.show()
save_plot(fig, 'comparisonLines_presentation', '5_1', False)

## Distances

In [ ]:
# get distances...
df = get_history_from_project('thesis-far-distances', set_name_index=False)
distances_df = df.copy()
distances_df['ID'] = distances_df.name.apply(lambda x: x.split("-")[0])
distances_df['OOD'] = distances_df.name.apply(lambda x: x.split("-")[1])
distances_df.set_index('name', drop=True, inplace=True)

# get aurocs
auroc_df = prep_far_ood('mls-1.0')
auroc_df.drop(['ID', 'OOD'], axis=1, inplace=True)

# get zsa... tip_df = get_history_from_project('thesis-tip-adapters-16_shots-test')
tip_df = get_history_from_project('thesis-tip-adapters-16_shots-test')

#add zsa to distances
distances_df['ID ZSA'] = distances_df['ID'].apply(lambda x: tip_df['ZEROSHOT'][x])
distances_df['OOD ZSA'] = distances_df['OOD'].apply(lambda x: tip_df['ZEROSHOT'][x])

# create full table
monstrum = pd.concat([auroc_df, distances_df], axis=1)

In [ ]:
monstrum.corr().max()

### only weak correlation

In [ ]:
monstrum.round(3).min()

In [ ]:
monstrum.round(3).max()

In [ ]:
monstrum[['MLS-1.0', 'mmd', 'clp', 'ID ZSA', 'OOD ZSA']].corr()['MLS-1.0']
save_csv(monstrum[['MLS-1.0', 'mmd', 'clp', 'ID ZSA', 'OOD ZSA']].corr().round(3)['MLS-1.0'], '5_1','far-corr-matrix')

In [ ]:
monstrum[(monstrum['MLS-1.0'] >0.98) & (monstrum['ID'].isin(['stanford cars','caltech cub']))][['MLS-1.0', 'mmd', 'clp']].corr()

### Deeper look into MSP 01

In [ ]:
working_df = get_history_from_project(frame_template.format('mls-1.0'), set_name_index=False)

working_df['ID'] = working_df.name.apply(lambda x: x.split("-")[0])
working_df['OOD'] = working_df.name.apply(lambda x: x.split("-")[1])

In [ ]:
working_df.groupby('ID').mean('AUROC').sort_values(by='AUROC', ascending=False)

## Perfectly over 95% on 67!!! combinations --> to easy in many settings or the method is very good

In [ ]:
working_df.max()

In [ ]:
# count by id
working_df[(working_df['AUROC'] > 0.99)].value_counts('ID')

In [ ]:
# count by od
working_df[(working_df['AUROC'] > 0.95)]

# Not working on: 5 with 50% +-5. 

In [ ]:
working_df[(working_df['AUROC'] <0.53) & (working_df['AUROC'] > 0.47)]

## Confused on only 5

In [ ]:
working_df[(working_df['AUROC'] <0.50)]

## Look into one run specifically

In [ ]:
mls1 = prep_far_ood('mls-1.0')

In [ ]:
mls1[(mls1['ID'] == 'cifar100') & (mls1['OOD'] == 'cifar10')]

In [ ]:
mls1[(mls1['ID'] == 'cifar10') & (mls1['OOD'] == 'cifar100')]

In [ ]:
mls1[(mls1['ID'] == 'cifar10') & (mls1['OOD'] == 'svhn')]